In [4]:
from Portfolio import Portfolio, universe # universe is a global variable
# and each portfolio can access to the universe

In [5]:
universe

In [6]:
instrumentsWeights = {"FPMB.F":0.5, "DCO.F":0.5}
portfolio_test = Portfolio(instrumentsWeights)
portfolio_test